This notebook enters the travel demand into the travel_demand sheet. to be improved following Eric's work

In [2]:
import pandas as pd

In [3]:
travel_demand = pd.read_excel(f'./TEAM-Kenya_data_out/Travel_demand.xlsx')
travel_demand

,ScenarioID,CountryID,PolicyID,Year,Trips_Com,chTrips_Com_Ageing,chTrips_Com_TeleWork,chTrips4dayweek,chTripsComGig,ATL_Com,...,td_all,OR_com,OR_busi,OR_leilong,OR_leiloc,OR_school,OR_shop,OR_pb,OR_other,OR_avg
0,1,9,0,2012,147.123993,1.000000,1.000000,1.000,1.000000,8.900050,...,6607.292969,1.117497,1.098954,2.046046,2.046046,1.914174,1.600436,1.587386,1.536749,1.553013
1,1,9,0,2013,145.859680,0.999591,1.000000,1.000,0.991812,8.771302,...,6637.943848,1.117497,1.098954,2.046049,2.046049,1.914177,1.600437,1.587388,1.536751,1.557800
2,1,9,0,2014,148.197998,0.999079,1.000000,1.000,1.008229,8.697289,...,6508.682617,1.117497,1.098954,2.046050,2.046049,1.914177,1.600437,1.587388,1.536751,1.547847
3,1,9,0,2015,143.379272,0.998720,1.000000,1.000,0.975796,9.116610,...,6654.992676,1.117497,1.098954,2.046050,2.046049,1.914177,1.600437,1.587388,1.536751,1.556540
4,1,9,0,2016,144.197952,0.998413,1.000000,1.000,0.981670,8.971702,...,6615.688965,1.117497,1.098954,2.046049,2.046049,1.914177,1.600437,1.587388,1.536751,1.553172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,1,9,0,2096,123.022781,0.941449,0.927835,0.975,0.981816,8.618203,...,6646.553711,1.132807,1.111848,2.182353,2.182353,2.033296,1.678676,1.663926,1.606691,1.679334
85,1,9,0,2097,122.999283,0.941269,0.927835,0.975,0.981816,8.618203,...,6647.082031,1.132807,1.111848,2.182353,2.182353,2.033296,1.678676,1.663926,1.606691,1.679399
86,1,9,0,2098,122.978004,0.941106,0.927835,0.975,0.981816,8.618203,...,6647.561523,1.132807,1.111848,2.182353,2.182353,2.033296,1.678676,1.663926,1.606691,1.679458
87,1,9,0,2099,122.958557,0.940957,0.927835,0.975,0.981816,8.618203,...,6648.000488,1.132807,1.111848,2.182353,2.182353,2.033296,1.678676,1.663926,1.606691,1.679512


In [4]:
# procesed travel survey data from WB survey (commuting journeys only

travel_survey_data = {'Walk': 19335, 'Microbus/Matatu': 5845, 'Bodaboda (Bicycle taxi)': 1092, 'Other, specify': 681, 'Own bicycle': 418, 'Own vehicle': 338, 'Bus regular': 199, 'Taxi (vehicle)': 51, 'Shared taxi': 1}

# conversions from modes in uk data to kenya data
conversions = {'Walk':'Walk', 'Bike':'Own bicycle', 'CarDriver':'Own vehicle',
               'CarPass':'Shared taxi', 'Moto': 'Bodaboda (Bicycle taxi)',
               'LocalBus':'Microbus/Matatu', 'Coach':'Bus regular',
               'RailTube':'Other, specify', 'OtherPriv':'Other, specify',
               'Taxi':'Taxi (vehicle)', 'Air':'Shared taxi', 'OtherPub': 'Other, specify'}

In [5]:
# list purposes (purps) and modes
purps = ['Com', 'Busi', 'LeiLong', 'LeiLoc', 'School', 'Shop', 'PB', 'Other']
modes = ['Walk', 'Bike', 'CarDriver', 'CarPass', 'Moto', 'LocalBus', 'Coach', 'RailTube',
         'OtherPriv', 'Taxi', 'Air', 'OtherPub']

# dictionary of scales (UK --> Kenya)
# TODO: test these assumptions with what Eric comes up with
scale_purps = {'Com':{'Trips':0.7, 'ATL':1}, 'Busi':{'Trips':0.7, 'ATL':1}, 'LeiLong':{'Trips':0.2, 'ATL':0.6}, 'LeiLoc':{'Trips':0.3, 'ATL':1}, 'School':{'Trips':1, 'ATL':1.2}, 'Shop':{'Trips':1, 'ATL':1}, 'PB':{'Trips':0.7, 'ATL':1}, 'Other':{'Trips':0.7, 'ATL':1}}

# enter number of trips and ATL based on scaling factors above.
for purp in purps:
    travel_demand.at[0,f'Trips_{purp}'] *= scale_purps[purp]['Trips']
    travel_demand.at[0,f'ATL_{purp}'] *= scale_purps[purp]['ATL']

In [6]:
# derive mode splits based on the WB travel survey data

mode_splits = {}
for purp in purps:

    mode_splits[purp] = {}

    for mode in modes:

        mode_splits[purp][mode] = travel_survey_data[conversions[mode]]/sum(travel_survey_data.values())

    # normalise to ensure that they add up to 1 at this level...
    factor=1.0/sum(mode_splits[purp].values())
    mode_splits[purp] = {k: v*factor for k, v in mode_splits[purp].items()}

In [7]:
mode_splits

{'Com': {'Walk': 0.6593800088667598,
  'Bike': 0.014255021655355861,
  'CarDriver': 0.011526787845718376,
  'CarPass': 3.410292262046857e-05,
  'Moto': 0.03724039150155168,
  'LocalBus': 0.19933158271663878,
  'Coach': 0.006786481601473245,
  'RailTube': 0.023224090304539095,
  'OtherPriv': 0.023224090304539095,
  'Taxi': 0.001739249053643897,
  'Air': 3.410292262046857e-05,
  'OtherPub': 0.023224090304539095},
 'Busi': {'Walk': 0.6593800088667598,
  'Bike': 0.014255021655355861,
  'CarDriver': 0.011526787845718376,
  'CarPass': 3.410292262046857e-05,
  'Moto': 0.03724039150155168,
  'LocalBus': 0.19933158271663878,
  'Coach': 0.006786481601473245,
  'RailTube': 0.023224090304539095,
  'OtherPriv': 0.023224090304539095,
  'Taxi': 0.001739249053643897,
  'Air': 3.410292262046857e-05,
  'OtherPub': 0.023224090304539095},
 'LeiLong': {'Walk': 0.6593800088667598,
  'Bike': 0.014255021655355861,
  'CarDriver': 0.011526787845718376,
  'CarPass': 3.410292262046857e-05,
  'Moto': 0.03724039150

In [8]:
for purp in purps:

    for mode in modes:
        travel_demand.at[0,f'MS_{purp}_{mode}'] = mode_splits[purp][mode]

travel_demand

,ScenarioID,CountryID,PolicyID,Year,Trips_Com,chTrips_Com_Ageing,chTrips_Com_TeleWork,chTrips4dayweek,chTripsComGig,ATL_Com,...,td_all,OR_com,OR_busi,OR_leilong,OR_leiloc,OR_school,OR_shop,OR_pb,OR_other,OR_avg
0,1,9,0,2012,102.986795,1.000000,1.000000,1.000,1.000000,8.900050,...,6607.292969,1.117497,1.098954,2.046046,2.046046,1.914174,1.600436,1.587386,1.536749,1.553013
1,1,9,0,2013,145.859680,0.999591,1.000000,1.000,0.991812,8.771302,...,6637.943848,1.117497,1.098954,2.046049,2.046049,1.914177,1.600437,1.587388,1.536751,1.557800
2,1,9,0,2014,148.197998,0.999079,1.000000,1.000,1.008229,8.697289,...,6508.682617,1.117497,1.098954,2.046050,2.046049,1.914177,1.600437,1.587388,1.536751,1.547847
3,1,9,0,2015,143.379272,0.998720,1.000000,1.000,0.975796,9.116610,...,6654.992676,1.117497,1.098954,2.046050,2.046049,1.914177,1.600437,1.587388,1.536751,1.556540
4,1,9,0,2016,144.197952,0.998413,1.000000,1.000,0.981670,8.971702,...,6615.688965,1.117497,1.098954,2.046049,2.046049,1.914177,1.600437,1.587388,1.536751,1.553172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,1,9,0,2096,123.022781,0.941449,0.927835,0.975,0.981816,8.618203,...,6646.553711,1.132807,1.111848,2.182353,2.182353,2.033296,1.678676,1.663926,1.606691,1.679334
85,1,9,0,2097,122.999283,0.941269,0.927835,0.975,0.981816,8.618203,...,6647.082031,1.132807,1.111848,2.182353,2.182353,2.033296,1.678676,1.663926,1.606691,1.679399
86,1,9,0,2098,122.978004,0.941106,0.927835,0.975,0.981816,8.618203,...,6647.561523,1.132807,1.111848,2.182353,2.182353,2.033296,1.678676,1.663926,1.606691,1.679458
87,1,9,0,2099,122.958557,0.940957,0.927835,0.975,0.981816,8.618203,...,6648.000488,1.132807,1.111848,2.182353,2.182353,2.033296,1.678676,1.663926,1.606691,1.679512


Now update the td_{mode} and td_{purp} values for the base year at the RHS of the table.

For td_{mode}, update using the mode_shares_triplength table.

For td_{purp}, simply use the existing table for ATL_{purp}*Trips{purp}

In [9]:
# load in mode_shares_triplength
mode_shares_triplength = pd.read_csv('./TEAM-Kenya_data_out/mode_shares_triplength_team_kenya.csv')
mode_shares_triplength

,ScenarioID,CountryID,PolicyID,Year,triplenID,modeID,td
0,1,9,0,2012,1,1,140.554895
1,1,9,0,2012,1,2,1.527704
2,1,9,0,2012,1,3,0.621154
3,1,9,0,2012,1,4,0.372925
4,1,9,0,2012,1,5,0.057659
...,...,...,...,...,...,...,...
8539,1,9,0,2100,8,8,11.078624
8540,1,9,0,2100,8,9,6.033094
8541,1,9,0,2100,8,10,2.257127
8542,1,9,0,2100,8,11,1.408617


In [10]:
# mode ID lookup for mode_shares_triplength table
modeID_lookup = {'Walk':1, 'Bike':2, 'CarDriver':3, 'CarPass':4, 'Moto':5, 'LocalBus':6, 'Coach':7, 'RailTube':8,
         'OtherPriv':9, 'Taxi':10, 'Air':11, 'OtherPub':12}

# ffs why is nothing in team the same. lookup between modes in the left of the table and the right of the table
cols_lookup_mode = {'Walk':'walk', 'Bike':'bike', 'CarDriver':'card', 'CarPass':'carp', 'Moto':'moto', 'LocalBus':'bus', 'Coach':'coach', 'RailTube':'rail', 'OtherPriv':'opriv', 'Taxi':'taxi', 'Air':'air', 'OtherPub':'opub'}

cols_lookup_purp = {'Com':'com', 'Busi':'busi', 'LeiLong':'leilong', 'LeiLoc':'leiloc', 'School':'school', 'Shop':'shop', 'PB':'pb', 'Other':'other'}

In [11]:
mode_shares_triplength[(mode_shares_triplength.modeID == 5) & (mode_shares_triplength.Year == 2012)].td.sum()

57.85162779316307

In [12]:
# update mode data
for mode in modes:

    # sum total distance for each mode across all triplengths
    td_mode = mode_shares_triplength[(mode_shares_triplength.modeID == modeID_lookup[mode]) & (mode_shares_triplength.Year == 2012)].td.sum()

    travel_demand.at[0, f'td_{cols_lookup_mode[mode]}'] = td_mode

In [13]:
# update purp data
for purp in purps:

    # multiply atl of purp by number of trips of purp
    td_purp = travel_demand.at[0, f'Trips_{purp}'] * travel_demand.at[0, f'ATL_{purp}']

    travel_demand.at[0, f'td_{cols_lookup_purp[purp]}'] = td_purp

In [14]:
# finally, sum all purp td values to give td_all
travel_demand.at[0, 'td_all'] = travel_demand.loc[0,['td_'+ p for p in list(cols_lookup_purp.values())]].sum()

In [17]:
travel_demand

,ScenarioID,CountryID,PolicyID,Year,Trips_Com,chTrips_Com_Ageing,chTrips_Com_TeleWork,chTrips4dayweek,chTripsComGig,ATL_Com,...,td_all,OR_com,OR_busi,OR_leilong,OR_leiloc,OR_school,OR_shop,OR_pb,OR_other,OR_avg
0,1,9,0,2012,102.986795,1.000000,1.000000,1.000,1.000000,8.900050,...,3708.318317,1.117497,1.098954,2.046046,2.046046,1.914174,1.600436,1.587386,1.536749,1.553013
1,1,9,0,2013,145.859680,0.999591,1.000000,1.000,0.991812,8.771302,...,6637.943848,1.117497,1.098954,2.046049,2.046049,1.914177,1.600437,1.587388,1.536751,1.557800
2,1,9,0,2014,148.197998,0.999079,1.000000,1.000,1.008229,8.697289,...,6508.682617,1.117497,1.098954,2.046050,2.046049,1.914177,1.600437,1.587388,1.536751,1.547847
3,1,9,0,2015,143.379272,0.998720,1.000000,1.000,0.975796,9.116610,...,6654.992676,1.117497,1.098954,2.046050,2.046049,1.914177,1.600437,1.587388,1.536751,1.556540
4,1,9,0,2016,144.197952,0.998413,1.000000,1.000,0.981670,8.971702,...,6615.688965,1.117497,1.098954,2.046049,2.046049,1.914177,1.600437,1.587388,1.536751,1.553172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,1,9,0,2096,123.022781,0.941449,0.927835,0.975,0.981816,8.618203,...,6646.553711,1.132807,1.111848,2.182353,2.182353,2.033296,1.678676,1.663926,1.606691,1.679334
85,1,9,0,2097,122.999283,0.941269,0.927835,0.975,0.981816,8.618203,...,6647.082031,1.132807,1.111848,2.182353,2.182353,2.033296,1.678676,1.663926,1.606691,1.679399
86,1,9,0,2098,122.978004,0.941106,0.927835,0.975,0.981816,8.618203,...,6647.561523,1.132807,1.111848,2.182353,2.182353,2.033296,1.678676,1.663926,1.606691,1.679458
87,1,9,0,2099,122.958557,0.940957,0.927835,0.975,0.981816,8.618203,...,6648.000488,1.132807,1.111848,2.182353,2.182353,2.033296,1.678676,1.663926,1.606691,1.679512


In [16]:
# write to csv
travel_demand.to_csv(f'./TEAM-Kenya_data_out/Travel_demand_TEAM-Kenya.csv', index=False)